# Chapter 8. Dimensionality Reduction
___

: 대부분의 Machine Learning 문제들은 각각의 training instance 마다 매우 많은 특성들(features)을 가지고 있다.

이와같은 데이터들은 training의 속도를 매우 늦출뿐 아니라, 좋은 solution을 찾는 것 또한 더욱 어렵게 만든다.

이러한 형상은 '***curse of dimensionality***' 라고 언급된다.

다행히도 실제 문제에서는 dimension의 수를 상당히 줄이는 것이 종종 가능하여 난해한 문제를 다루기 쉬운 문제로 전환시키기도 한다. (e.g. MNIST images)

<***Warning***>

> Dimensionality를 줄이는 것은 몇몇의 정보를 잃는 것과 같다. 그러므로 이는 training의 속도를 올려주지만 약간 system 의 기능을 낮춘다. </br>
> 그리고 이는 piplines을 약간 더 복잡하게 만들고 유지하기 어렵게 만든다. </ㅠㄱ>
> **그러므로 training이 너무 느려서 *Dimensionality Reduction*을 고려해보기 이전에, *먼저 original data를 통해 system을 학습*시켜보아야 한다.** </br>
> 몇몇의 경우에서는 training data의 dimensionality를 줄이는 것은 noise나 불필요한 정보들을 걸러줄 수 도있고, 더 좋은 결과를 보이기도 한다. </br>
> 하지만, **보통의 경우에서는 좋은 결과를 내지는 않는다. 그저 속도만 높여준다.**

: training을 더 빠르게 할 수 있는다는 것과는 별개로, **dimensionality reduction**은 ***data visualization (DataViz)***에 아주 유용하다.

    - dimension들의 수를 2~3정도로 줄이는 것은 고차원 training set을 graph로 plot 할 수 있고, cluster와 같은 pattern을 시각적으로 감지하여 몇 가지 중요한 관점을 얻는 경우가 많다.
    
이번 장에서는 데이터의 차원을 축소하는 방법인 투영(projection) 과 manifold learning 그리고 대표적인 dimensionality algorithm인 ' ***주 성분 분석 (PCA), Kernal PCA, LLE*** ' 등에 대해 알아볼 것이다.

</br>

___

## 8.1 The Curse of Dimensionality

머신러닝에서 데이터 셋의 특성(feature)가 많아지면, 각 특성인 하나의 차원(dimension) 또한 증가하게 된다. 

이렇게 데이터의 차원이 증가할 수록 데이터 공간의 부피가 기하 급수적으로 증가하기 때문에, 데이터의 밀도는 차원이 증가할 수록 희소(sparse)해진다. 

아래의 그림은 데이터의 차원이 증가할수록 각 영역(region)에 포함되는 데이터의 개수를 나타낸 그림(출처: jermwatt's blog)이다. 

그림에서 알 수 있듯이 차원이 증가할 수록 데이터의 밀도가 희소해지는것을 알 수 있다.

![img_1](./imgs/8-1-1.png)

데이터의 차원이 증가할수록 데이터 포인트 간의 거리 또한 증가하게 되므로, 이러한 데이터를 이용해 머신러닝 알고리즘을 학습 하게되면 모델이 복잡해지게 된다. 

따라서, **오버피팅(overfitting)** 위험이 커진다. 

이러한 curse of dimensionality를 해결하기 위한 방법 중 하나는 데이터의 밀도가 높아질 때까지 train dataset을 늘리는 것이다. 

하지만, dataset의 크기에 비해 dimension은 기하급수적으로 커지기 때문에 매우 힘든일이라 할 수 있다.


</br>

___

## 8.2 Main Approaches for Dimensionality Reduction

: Dimensionality Reduction의 두 가지 접근 방법인 '**Projection**' 과 '**Manifold Learning**'에 대해 알아볼 것이다

</br>

### 8.2.1 Projection(투영)

: 일반적으로 대부분의 real-world problems 에서 모든 데이터의 features, 즉 dimension이 고르게 분포되어 있지 않다. 

데이터셋인 MNIST를 예로들면, 어떤 특성(각 pixel을 하나의 feature로 볼 때)은 거의 변화가 없고, 또 어떤 feature는 다른 feature들과 서로 연관되어 있다. 

이렇듯 학습 데이터셋은 고차원 공간에서 저차원 부분 공간(subspace)에 위치하게 된다. 

즉, 고차원의 데이터의 feature 중 일부 feature으로 데이터를 표현할 수 있다는 말이 된다.

![img_2](./imgs/8-2-1.PNG)

위의 그림은 왼쪽의 3차원 공간상의 데이터를 2차원 subspace로 projection 시켜, 2차원 dataset으로 만든 것이다.

그러나, projection은 항상 좋은 방법은 아니다. 

</br>

-> 몇몇의 경우에는 subspace가 꼬이거나 뒤집혀 있는데 이런 경우를 ***Swiss roll*** toy dataset 이라 부르며 아래의 그림과 같다.

![img_2](./imgs/8-2-2.PNG)

이러한 유형의 dataset을 그저 평면으로 projecting 시킨다면, 왼쪽 아래와 같이 다른 층들간에 충돌이 일어난다. 

그러나, 우리가 원하는 것은 오른쪽 아래의 그림처럼 datasetdl unroll 되는것을 원한다.

![img_2](./imgs/8-2-3.PNG)

</br>

___

### 8.2.2 Manifold Learning

#### < 정의 >

> ![img_5](./imgs/8-2-4.PNG) </br>
> manifold는 다양체라고도 하며, 국소적으로 유클리드 공간과 닮은 위상 공간이다. </br> 즉, 국소적으로는 유클리드 공간과 구별할 수 없으나, 대역적으로 독특한 위상수학적 구조를 가질 수 있다 (출처 : 위키피디아). </br> 예를 들어, 아래의 원 그림은 모든 점에 대해서 국소적으로 직선과 같은 구조를 갖는 1차원 manifold이다. </br>


</br>

아래의 그림은 **Swis roll** dataset이며, 2D-manifold의 예시로 쓰인다. 

그림에서 볼 수 있듯 2D-manifold는 고차원(3D) 공간에서 휘거나 말린 2D 모양이다. 

일반적으로 d-차원 manifold는 국소적으로 d-차원 초평면으로 볼 수 있는 *n*-차원 공간의 일부이다. (*d < n*)

Swiss roll은 *d* = 2 이고, *n* = 3 인 국소적으로는 2D 평면이지만, 3차원으로 말려있는 dataset이다.

![img_6](./imgs/8-2-5.PNG)

</br>

대부분의 **Dimensionality Reduction** algorithm은 이러한 manifold를 modeling하는 방식으로 동작하며, 이를 **Manifold Learning** 이라고 한다.

이 Manifold Learning은 **Manifold assumption(매니폴드 가정)** 또는 **Manifold hypothesis** 에 의해, 고차원인 실제 dataset이 더 낮은 저차원 manifold에 가깝게 놓여 있다고 가정한다.

**Manifold assumption** 은 종종 다른 가정과 같이 쓰인다.

예를들어, classifier 나 Regression 과 같은 작업을 하기전에 training dataset을 저차원의 manifold 공산으로 표현하면 더 간단하게 문제를 해결 할 수 있다는 가정을 할 수 있다.

![img_7](./imgs/8-2-6.PNG)

</br>

하지만, 위와같은 가정이 항상 통하지 않는다. 

아래의 그림처럼 저차원의 manifold가 오히려 ***Decision Boundary***를 찾는 것이 더 어려운 것을 알 수 있다.

![img_8](./imgs/8-2-7.PNG)

</br>

따라서 model을 학습시키기 전에 training dataset의 차원을 감소시키면, 학습속도는 빨라지지만 model의 성능은 항상 더 나아지거나 더 간단한 model이 되는것은 아니다. 

이는 dataset의 형태에 따라 달라진다.

</br>

___

## 8.3 PCA (Principal Component Analysis)

### < Reference >

책 : Hands on Machine Learning

code 및 images : https://excelsior-cjh.tistory.com/167?category=918734 [EXCELSIOR]